In [1]:
import deepmatcher as dm
import torch

ImportError: cannot import name 'AttrTensor'

In [2]:
train, validation, test = dm.process(
    path='~/emdata/structured_data_rm_long_attrs/Amazon-Google/',
    train='train_dm.csv',
    validation='valid_dm.csv',
    test='test_dm.csv',
    ignore_columns=('left_id', 'right_id'),
    include_lengths=True,
    auto_rebuild_cache=True,
    pca=True)

In [3]:
def my_attr_comparator(left, right):
    return torch.cat((left,
                      right,
                      torch.abs(left - right),
                      left * right), left.dim() - 1)

custom_model = dm.MatchingModel(
    attr_summarizer=dm.attr_summarizers.Hybrid(
        word_contextualizer=dm.word_contextualizers.RNN(unit_type='LSTM',
                                                        layers=2,
                                                        dropout=0.2,
                                                        bypass_network='highway'),
        word_comparator=dm.word_comparators.Attention(
            input_dropout=0.3,
            alignment_network=dm.modules.AlignmentNetwork(
                    style='concat_dot',
                    transform_network='2-layer-residual-tanh'),
            comparison_network=dm.modules.Transform('3-layer-highway',
                                                    hidden_size=400,
                                                    output_size=200)),
        word_aggregator=dm.word_aggregators.AttentionWithRNN(
            input_dropout=0.2,
            alignment_network='decomposable',
            rnn=dm.modules.RNN(unit_type='GRU',
                               layers=3,
                               dropout=0.1,
                               bypass_network='residual'),
            rnn_pool_style='max')),
    attr_comparator=lambda: dm.modules.Lambda(my_attr_comparator),
    classifier=dm.Classifier(transform_network='1-layer-residual-glu',
                             hidden_size=300),
    finetune_embeddings=True)


In [4]:
custom_model.run_train(
    train,
    validation,
    label_smoothing=0.05,
    batch_size=16,
    epochs=10,
    best_save_path='/tmp/custom_model.pth',
    pos_weight=1.3)

* Number of trainable parameters: 13419155
=>  TRAIN epoch 1 :


HBox(children=(IntProgress(value=0, max=86), HTML(value='')))


Finished Epoch 1 || Run Time: 53.581628 | Load Time: 2.511045 | F1:    9.60 | Prec:   40.86 | Rec:    5.44 || Ex/s:  122.5

=>  EVAL epoch 1 :


HBox(children=(IntProgress(value=0, max=28), HTML(value='')))


Finished Epoch 1 || Run Time: 7.433477 | Load Time: 0.809422 | F1:   12.45 | Prec:   43.59 | Rec:    7.26 || Ex/s:  278.2

* Best F1: 12.454212454212453
Saving best model...
=>  TRAIN epoch 2 :


...


Finished Epoch 9 || Run Time: 7.386516 | Load Time: 0.812608 | F1:   40.40 | Prec:   37.97 | Rec:   43.16 || Ex/s:  279.6

* Best F1: 40.4
Saving best model...
=>  TRAIN epoch 10 :


HBox(children=(IntProgress(value=0, max=86), HTML(value='')))


Finished Epoch 10 || Run Time: 54.124701 | Load Time: 2.642583 | F1:   80.76 | Prec:   82.96 | Rec:   78.68 || Ex/s:  121.1

=>  EVAL epoch 10 :


HBox(children=(IntProgress(value=0, max=28), HTML(value='')))


Finished Epoch 10 || Run Time: 7.409412 | Load Time: 0.812097 | F1:   40.33 | Prec:   39.27 | Rec:   41.45 || Ex/s:  278.9



In [5]:
custom_model.load_state('/tmp/custom_model.pth')
custom_model.run_eval(test)

=>  EVAL epoch 9 :


HBox(children=(IntProgress(value=0, max=14), HTML(value='')))


Finished Epoch 9 || Run Time: 3.775275 | Load Time: 0.781406 | F1:   37.27 | Prec:   35.09 | Rec:   39.74 || Ex/s:  503.1



37.27454909819639